# goal: standardize one csv file from bellevue

(and maybe mayo clinic)
source: 
https://hhinternet.blob.core.windows.net/machine-readable/132655001_Bellevue%20Hospital%20Center_standardcharges.csv 
or https://data.payless.health/

- [ ] ask GPT-4 to take the docs and turn it into a parquet file
- [ ] ask claude
- [ ] try some columns
- [ ] upload it to s3
- [ ] then try to change the observable link for a demo


# Bellevue

In [1]:
!head  /Users/me/Downloads/132655001_Bellevue\ Hospital\ Center_standardcharges.csv

Procedure,Code Type,Code,NDC,Rev Code,Procedure Description,Payer,Plan(s),IP Price,OP Price,Discounted Cash Price,IP Negotiated Charge,OP Negotiated Charge,PB Negotiated Charge,De-Identified Minimum Negotiated Charge (IP),De-Identified Maximum Negotiated Charge (IP),De-Identified Minimum Negotiated Charge (OP),De-Identified Maximum Negotiated Charge (OP)
001573,SUP,,,0272 - MEDICAL/SURGICAL SUPPLIES AND DEVICES - STERILE SUPPLY,CEMENT DENTAL GC RADIOPAQUE PASTE PAK DISPENSER( - 001573),HEALTHFIRST MCD [3360],HEALTHFIRST MEDICAID MC [336000],95.84,95.84,38.34,Not reimbursed separately,External grouping not supported,Not Reimbursed Separately,Not Available,Not Available,Not Available,Not Available
001573,SUP,,,0272 - MEDICAL/SURGICAL SUPPLIES AND DEVICES - STERILE SUPPLY,CEMENT DENTAL GC RADIOPAQUE PASTE PAK DISPENSER( - 001573),HEALTHFIRST MCR [2340],HEALTHFIRST LIFE IMPROVEMENT HMO SNP [234000],95.84,95.84,38.34,Not reimbursed separately,APC grouping requires price history,Not Reimburs

In [2]:
# Load duckdb, which lets us efficiently load large files
import duckdb

# Load pandas, which lets us manipulate dataframes
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True

%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Connect jupysql to DuckDB using a SQLAlchemy-style connection string. Either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:

# Prompt for reading in the csv file

```quote

DuckDB Logo for Download
Documentation

Blog
GitHub
Contributing

    Installation
    Guides
    Documentation
        Connect
        Data Import
            Overview
            CSV Files
                Overview
                Auto Detection
                Tips
            JSON Files
            Multiple Files
            Parquet Files
            Partitioning
            Appender
            Insert Statements
        Client APIs
        SQL
            Introduction
            Statements
            Query Syntax
            Data Types
            Expressions
            Functions
            Aggregates
            Configuration
            Constraints
            Indexes
            Information Schema
            Metadata Functions
            Pragmas
            Samples
            Window Functions
        Extensions
    Development
    Sitemap
    Why DuckDB
    FAQ
    Code of Conduct
    Live Demo

CSV Loading
Version dev
Version:

    dev

Examples

-- read a CSV file from disk, auto-infer options
SELECT * FROM 'flights.csv';
-- read_csv with custom options
SELECT * FROM read_csv('flights.csv', delim='|', header=True, columns={'FlightDate': 'DATE', 'UniqueCarrier': 'VARCHAR', 'OriginCityName': 'VARCHAR', 'DestCityName': 'VARCHAR'});
-- read a CSV from stdin, auto-infer options
cat data/csv/issue2471.csv | duckdb -c "select * from read_csv_auto('/dev/stdin')"

-- read a CSV file into a table
CREATE TABLE ontime(FlightDate DATE, UniqueCarrier VARCHAR, OriginCityName VARCHAR, DestCityName VARCHAR);
COPY ontime FROM 'flights.csv' (AUTO_DETECT TRUE);
-- alternatively, create a table without specifying the schema manually
CREATE TABLE ontime AS SELECT * FROM 'flights.csv';

-- write the result of a query to a CSV file
COPY (SELECT * FROM ontime) TO 'flights.csv' WITH (HEADER 1, DELIMITER '|');

CSV Loading

CSV loading is a very common, and yet surprisingly tricky, task. While CSVs seem simple on the surface, there are a lot of inconsistencies found within CSV files that can make loading them a challenge. CSV files come in many different varieties, are often corrupt, and do not have a schema. The CSV reader needs to cope with all of these different situations.

The DuckDB CSV reader can automatically infer which configuration flags to use by analyzing the CSV file. This will work correctly in most situations, and should be the first option attempted. In rare situations where the CSV reader cannot figure out the correct configuration it is possible to manually configure the CSV reader to correctly parse the CSV file. See the auto detection page for more information.

Below are parameters that can be passed in to the CSV reader.
Parameters
Name 	Description 	Type 	Default
all_varchar 	Option to skip type detection for CSV parsing and assume all columns to be of type VARCHAR. 	bool 	false
auto_detect 	Enables auto detection of parameters 	bool 	true
columns 	A struct that specifies the column names and column types contained within the CSV file (e.g. {'col1': 'INTEGER', 'col2': 'VARCHAR'}). 	struct 	(empty)
compression 	The compression type for the file. By default this will be detected automatically from the file extension (e.g. t.csv.gz will use gzip, t.csv will use none). Options are none, gzip, zstd. 	varchar 	auto
dateformat 	Specifies the date format to use when parsing dates. See Date Format 	varchar 	(empty)
decimal_separator 	The decimal separator of numbers 	varchar 	.
delim or sep 	Specifies the string that separates columns within each row (line) of the file. 	varchar 	,
escape 	Specifies the string that should appear before a data character sequence that matches the quote value. 	varchar 	"
filename 	Whether or not an extra filename column should be included in the result. 	bool 	false
force_not_null 	Do not match the specified columns’ values against the NULL string. In the default case where the NULL string is empty, this means that empty values will be read as zero-length strings rather than NULLs. 	varchar[] 	[]
header 	Specifies that the file contains a header line with the names of each column in the file. 	bool 	false
hive_partitioning 	Whether or not to interpret the path as a hive partitioned path. 	bool 	false
ignore_errors 	Option to ignore any parsing errors encountered - and instead ignore rows with errors. 	bool 	false
max_line_size 	The maximum line size in bytes 	bigint 	2097152
names 	The column names as a list. Example here. 	varchar[] 	(empty)
new_line 	Set the new line character(s) in the file. Options are '\r','\n', or '\r\n'. 	varchar 	(empty)
normalize_names 	Boolean value that specifies whether or not column names should be normalized, removing any non-alphanumeric characters from them. 	bool 	false
nullstr 	Specifies the string that represents a NULL value. 	varchar 	(empty)
parallel 	Whether or not the experimental parallel CSV reader is used. 	bool 	false
quote 	Specifies the quoting string to be used when a data value is quoted. 	varchar 	"
sample_size 	The number of sample rows for auto detection of parameters. 	bigint 	20480
skip 	The number of lines at the top of the file to skip. 	bigint 	0
timestampformat 	Specifies the date format to use when parsing timestamps. See Date Format 	varchar 	(empty)
types or dtypes 	The column types as either a list (by position) or a struct (by name). Example here. 	varchar[] or struct 	(empty)
union_by_name 	Whether the columns of multiple schemas should be unified by name, rather than by position. 	bool 	false
Writing

The contents of tables or the result of queries can be written directly to a CSV file using the COPY statement. See the COPY documentation for more information.
read_csv_auto function

The read_csv_auto is the simplest method of loading CSV files: it automatically attempts to figure out the correct configuration of the CSV reader. It also automatically deduces types of columns. If the CSV file has a header, it will use the names found in that header to name the columns. Otherwise, the columns will be named column0, column1, column2, ...

SELECT * FROM read_csv_auto('flights.csv');

FlightDate 	UniqueCarrier 	OriginCityName 	DestCityName
1988-01-01 	AA 	New York, NY 	Los Angeles, CA
1988-01-02 	AA 	New York, NY 	Los Angeles, CA
1988-01-03 	AA 	New York, NY 	Los Angeles, CA

The path can either be a relative path (relative to the current working directory) or an absolute path.

We can use read_csv_auto to create a persistent table as well:

CREATE TABLE ontime AS SELECT * FROM read_csv_auto('flights.csv');
DESCRIBE ontime;

Field 	Type 	Null 	Key 	Default 	Extra
FlightDate 	DATE 	YES 	NULL 	NULL 	NULL
UniqueCarrier 	VARCHAR 	YES 	NULL 	NULL 	NULL
OriginCityName 	VARCHAR 	YES 	NULL 	NULL 	NULL
DestCityName 	VARCHAR 	YES 	NULL 	NULL 	NULL

SELECT * FROM read_csv_auto('flights.csv', SAMPLE_SIZE=20000);

If we set DELIM/SEP, QUOTE, ESCAPE, or HEADER explicitly, we can bypass the automatic detection of this particular parameter:

SELECT * FROM read_csv_auto('flights.csv', HEADER=TRUE);

Multiple files can be read at once by providing a glob or a list of files. Refer to the multiple files section for more information.
COPY Statement

The COPY statement can be used to load data from a CSV file into a table. This statement has the same syntax as the COPY statement supported by PostgreSQL. For the COPY statement, we must first create a table with the correct schema to load the data into. We then specify the CSV file to load from plus any configuration options separately.

CREATE TABLE ontime(flightdate DATE, uniquecarrier VARCHAR, origincityname VARCHAR, destcityname VARCHAR);
COPY ontime FROM 'flights.csv' ( DELIMITER '|', HEADER );
SELECT * FROM ontime;

flightdate 	uniquecarrier 	origincityname 	destcityname
1988-01-01 	AA 	New York, NY 	Los Angeles, CA
1988-01-02 	AA 	New York, NY 	Los Angeles, CA
1988-01-03 	AA 	New York, NY 	Los Angeles, CA

If we want to use the automatic format detection, we can set AUTO_DETECT to TRUE and omit the otherwise required configuration options.

CREATE TABLE ontime(flightdate DATE, uniquecarrier VARCHAR, origincityname VARCHAR, destcityname VARCHAR);
COPY ontime FROM 'flights.csv' ( AUTO_DETECT TRUE );
SELECT * FROM ontime;

More on the copy statement can be found here.

    Auto Detection
    Tips

Search Shortcut cmd + k | ctrl + k


%%sql
COPY (SELECT *
FROM read_csv('./data/cityofnewyork.us/311-Service-Requests-from-2010-to-Present.csv',
    header=True,
    delim=',',
    quote='"',
    columns={'Unique Key': 'BIGINT',
    'Created Date': 'VARCHAR',
    'Closed Date': 'VARCHAR',
    'Agency': 'VARCHAR',
    'Agency Name': 'VARCHAR',
    'Complaint Type': 'VARCHAR',
    'Descriptor': 'VARCHAR',
    'Location Type': 'VARCHAR',
    'Incident Zip': 'VARCHAR',
    'Incident Address': 'VARCHAR',
    'Street Name': 'VARCHAR',
    'Cross Street 1': 'VARCHAR',
    'Cross Street 2': 'VARCHAR',
    'Intersection Street 1': 'VARCHAR',
    'Intersection Street 2': 'VARCHAR',
    'Address Type': 'VARCHAR',
    'City': 'VARCHAR',
    'Landmark': 'VARCHAR',
    'Facility Type': 'VARCHAR',
    'Status': 'VARCHAR',
    'Due Date': 'VARCHAR',
    'Resolution Description': 'VARCHAR',
    'Resolution Action Updated Date': 'VARCHAR',
    'Community Board': 'VARCHAR',
    'BBL': 'VARCHAR',
    'Borough': 'VARCHAR',
    'X Coordinate (State Plane)': 'VARCHAR',
    'Y Coordinate (State Plane)': 'VARCHAR',
    'Open Data Channel Type': 'VARCHAR',
    'Park Facility Name': 'VARCHAR',
    'Park Borough': 'VARCHAR',
    'Vehicle Type': 'VARCHAR',
    'Taxi Company Borough': 'VARCHAR',
    'Taxi Pick Up Location': 'VARCHAR',
    'Bridge Highway Name': 'VARCHAR',
    'Bridge Highway Direction': 'VARCHAR',
    'Road Ramp': 'VARCHAR',
    'Bridge Highway Segment': 'VARCHAR',
    'Latitude': 'DOUBLE',
    'Longitude': 'DOUBLE',
    'Location': 'VARCHAR'}) 
LIMIT 1000000 -- uncomment this line to create a smaller version of the file for testing purposes
) TO './data/cityofnewyork.us/311-Service-Requests-from-2010-to-Present.parquet' (COMPRESSION ZSTD);

!head  /Users/me/Downloads/132655001_Bellevue\ Hospital\ Center_standardcharges.csv
Procedure,Code Type,Code,NDC,Rev Code,Procedure Description,Payer,Plan(s),IP Price,OP Price,Discounted Cash Price,IP Negotiated Charge,OP Negotiated Charge,PB Negotiated Charge,De-Identified Minimum Negotiated Charge (IP),De-Identified Maximum Negotiated Charge (IP),De-Identified Minimum Negotiated Charge (OP),De-Identified Maximum Negotiated Charge (OP)
001573,SUP,,,0272 - MEDICAL/SURGICAL SUPPLIES AND DEVICES - STERILE SUPPLY,CEMENT DENTAL GC RADIOPAQUE PASTE PAK DISPENSER( - 001573),HEALTHFIRST MCD [3360],HEALTHFIRST MEDICAID MC [336000],95.84,95.84,38.34,Not reimbursed separately,External grouping not supported,Not Reimbursed Separately,Not Available,Not Available,Not Available,Not Available
001573,SUP,,,0272 - MEDICAL/SURGICAL SUPPLIES AND DEVICES - STERILE SUPPLY,CEMENT DENTAL GC RADIOPAQUE PASTE PAK DISPENSER( - 001573),HEALTHFIRST MCR [2340],HEALTHFIRST LIFE IMPROVEMENT HMO SNP [234000],95.84,95.84,38.34,Not reimbursed separately,APC grouping requires price history,Not Reimbursed Separately,Not Available,Not Available,Not Available,Not Available
001573,SUP,,,0272 - MEDICAL/SURGICAL SUPPLIES AND DEVICES - STERILE SUPPLY,CEMENT DENTAL GC RADIOPAQUE PASTE PAK DISPENSER( - 001573),METROPLUS COMM [1002],METROPLUS GOLDPLUS [100202],95.84,95.84,38.34,Not reimbursed separately,APC grouping requires price history,Not Reimbursed Separately,Not Available,Not Available,Not Available,Not Available
001573,SUP,,,0272 - MEDICAL/SURGICAL SUPPLIES AND DEVICES - STERILE SUPPLY,CEMENT DENTAL GC RADIOPAQUE PASTE PAK DISPENSER( - 001573),UNITEDHEALTHCARE COMM [5090],UHC COMMUNITY PLAN CHP [509009],95.84,95.84,38.34,Not reimbursed separately,External grouping not supported,0.00,Not Available,Not Available,Not Available,Not Available
001-899120,SUP,,,0272 - MEDICAL/SURGICAL SUPPLIES AND DEVICES - STERILE SUPPLY,CYLINDER MEDICAL GAS NITROGEN 9% HELIUM 21% OXYGEN( - 001-899120),AFFINITY HEALTH PLAN MCD [3140],AFFINITY MEDICAID HMO [314000],126.14,126.14,50.46,Not reimbursed separately,External grouping not supported,Not Reimbursed Separately,Not Available,Not Available,Not Available,Not Available

please the above example and documentation to create a similar call to the read_csv function using the file alongside its header information
```

Result:

https://chat.openai.com/share/451ab1ef-2304-466f-8e3b-41f492780706 or below

In [48]:
%%sql

SELECT * 
FROM read_csv('/Users/me/Downloads/132655001_Bellevue Hospital Center_standardcharges.csv', 
              header=True, 
              delim=',', 
              quote='"', 
              nullstr='Not Available', --  Not reimbursed separately,External grouping not supported,Not Available,Not Applicable
              ignore_errors=true,
              columns={'Procedure': 'VARCHAR',
                       'Code Type': 'VARCHAR',
                       'Code': 'VARCHAR',
                       'NDC': 'VARCHAR',
                       'Rev Code': 'VARCHAR',
                       'Procedure Description': 'VARCHAR',
                       'Payer': 'VARCHAR',
                       'Plan(s)': 'VARCHAR',
                       'IP Price': 'DOUBLE',
                       'OP Price': 'DOUBLE',
                       'Discounted Cash Price': 'DOUBLE',
                       'IP Negotiated Charge': 'VARCHAR',
                       'OP Negotiated Charge': 'DOUBLE',
                       'PB Negotiated Charge': 'DOUBLE',
                       'De-Identified Minimum Negotiated Charge (IP)': 'DOUBLE',
                       'De-Identified Maximum Negotiated Charge (IP)': 'DOUBLE',
                       'De-Identified Minimum Negotiated Charge (OP)': 'DOUBLE',
                       'De-Identified Maximum Negotiated Charge (OP)': 'DOUBLE'
                      }) LIMIT 10;


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Procedure,Code Type,Code,NDC,Rev Code,Procedure Description,Payer,Plan(s),IP Price,OP Price,Discounted Cash Price,IP Negotiated Charge,OP Negotiated Charge,PB Negotiated Charge,De-Identified Minimum Negotiated Charge (IP),De-Identified Maximum Negotiated Charge (IP),De-Identified Minimum Negotiated Charge (OP),De-Identified Maximum Negotiated Charge (OP)
0,0020-00-0184-01,SUP,,,0272 - MEDICAL/SURGICAL SUPPLIES AND DEVICES -...,KIT MNTR ACTR + NIBP CUF REUSE( - 0020-00-0184...,UNITEDHEALTHCARE COMM [5090],UHC COMMERCIAL [509006],153.90,153.90,61.56,Not reimbursed separately,123.12,0.0,NaN,NaN,NaN,NaN
1,00236003632,SUP,,,0272 - MEDICAL/SURGICAL SUPPLIES AND DEVICES -...,GUIDEWIRE ORTHOPEDIC 3.2MM PERCUTANEOUS DRILL ...,UNITEDHEALTHCARE COMM [5090],UHC COMPASS [509000],67.50,67.50,27.00,Not reimbursed separately,54.00,0.0,NaN,NaN,NaN,NaN
2,003204,SUP,,,0278 - MEDICAL/SURGICAL SUPPLIES AND DEVICES -...,CLIP INTERNAL WECK HORIZON TITANIUM MEDIUM LAR...,UNITEDHEALTHCARE COMM [5090],UHC COMMERCIAL [509006],17.78,17.78,7.11,Not reimbursed separately,14.22,0.0,NaN,NaN,NaN,NaN
3,00340,SUP,,,0272 - MEDICAL/SURGICAL SUPPLIES AND DEVICES -...,SEALANT FSSR TUBLISEAL( - 00340),UNITEDHEALTHCARE COMM [5090],UHC OPTIONS [509010],65.71,65.71,26.28,Not reimbursed separately,52.57,0.0,NaN,NaN,NaN,NaN
4,00481204500,SUP,,,0272 - MEDICAL/SURGICAL SUPPLIES AND DEVICES -...,SCREWDRIVER LARGE HEXAGON STAINLESS STEEL 3.5M...,UNITEDHEALTHCARE COMM [5090],UHC COMPASS [509000],151.88,151.88,60.75,Not reimbursed separately,121.50,0.0,NaN,NaN,NaN,NaN
5,00481204500,SUP,,,0272 - MEDICAL/SURGICAL SUPPLIES AND DEVICES -...,SCREWDRIVER LARGE HEXAGON STAINLESS STEEL 3.5M...,UNITEDHEALTHCARE COMM [5090],UHC SELECT [509011],151.88,151.88,60.75,Not reimbursed separately,121.50,0.0,NaN,NaN,NaN,NaN
6,00490101215,SUP,,,0278 - MEDICAL/SURGICAL SUPPLIES AND DEVICES -...,WIRE FIXATION 150MM 1.25MM KIRSCHNER STAINLESS...,UNITEDHEALTHCARE COMM [5090],UHC COMPASS [509000],9.00,9.00,3.60,Not reimbursed separately,7.20,0.0,NaN,NaN,NaN,NaN
7,00490101615,SUP,,,0278 - MEDICAL/SURGICAL SUPPLIES AND DEVICES -...,WIRE FIXATION 1.6MM 150MM KIRSCHNER STAINLESS ...,UNITEDHEALTHCARE COMM [5090],UHC COMMERCIAL [509006],9.00,9.00,3.60,Not reimbursed separately,7.20,0.0,NaN,NaN,NaN,NaN
8,00504900100,SUP,,,0272 - MEDICAL/SURGICAL SUPPLIES AND DEVICES -...,BOWL BONE CEMENT QUICK-VAC LARGE CAPACITY VACU...,UNITEDHEALTHCARE COMM [5090],UHC EMPIRE [509012],73.75,73.75,29.50,Not reimbursed separately,73.75,0.0,NaN,NaN,NaN,NaN
9,00505024600,SUP,,,0278 - MEDICAL/SURGICAL SUPPLIES AND DEVICES -...,WIRE FX .028IN 5IN CWR SS TROC TIP( - 00505024...,UNITEDHEALTHCARE COMM [5090],UHC OPTIONS [509010],9.93,9.93,3.97,Not reimbursed separately,7.94,0.0,NaN,NaN,NaN,NaN


In [54]:
%%sql

COPY (SELECT *
FROM read_csv('/Users/me/Downloads/132655001_Bellevue Hospital Center_standardcharges.csv', 
              header=True, 
              delim=',', 
              quote='"', 
              ignore_errors=true,
              parallel=false,
              nullstr='Not Available', --  Not reimbursed separately,External grouping not supported,Not Available
              columns={'Procedure': 'VARCHAR',
                       'Code Type': 'VARCHAR',
                       'Code': 'VARCHAR',
                       'NDC': 'VARCHAR',
                       'Rev Code': 'VARCHAR',
                       'Procedure Description': 'VARCHAR',
                       'Payer': 'VARCHAR',
                       'Plan(s)': 'VARCHAR',
                       'IP Price': 'VARCHAR',
                       'OP Price': 'VARCHAR',
                       'Discounted Cash Price': 'DOUBLE',
                       'IP Negotiated Charge': 'VARCHAR',
                       'OP Negotiated Charge': 'VARCHAR',
                       'PB Negotiated Charge': 'VARCHAR',
                       'De-Identified Minimum Negotiated Charge (IP)': 'DOUBLE',
                       'De-Identified Maximum Negotiated Charge (IP)': 'DOUBLE',
                       'De-Identified Minimum Negotiated Charge (OP)': 'DOUBLE',
                       'De-Identified Maximum Negotiated Charge (OP)': 'DOUBLE'
                      })
) TO './data/data.payless.health/raw/132655001_Bellevue Hospital Center_standardcharges.parquet' (COMPRESSION ZSTD);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,4157401


## Testing SQL queries locally before Observable notebook



In [4]:
%%sql 

CREATE TABLE IF NOT EXISTS flights10m AS
    SELECT
      GREATEST(-60, LEAST(ARR_DELAY, 180))::DOUBLE AS delay,
      DISTANCE AS distance,
      DEP_TIME AS time
    FROM 'https://onefact.github.io/datathinking.org-codespace/data/data.payless.health/processed/132655001_Bellevue%20Hospital%20Center_standardcharges.parquet'

BinderException: Binder Error: Referenced column "ARR_DELAY" not found in FROM clause!
Candidate bindings: "132655001_Bellevue%20Hospital%20Center_standardcharges.Code"
LINE 3:       GREATEST(-60, LEAST(ARR_DELAY, 180))::DOUBLE AS delay,
                                  ^

In [12]:
%%sql 

CREATE TABLE IF NOT EXISTS healthCharges AS
    SELECT
      -- GREATEST('De-Identified Maximum Negotiated Charge (IP)', 'De-Identified Minimum Negotiated Charge (IP)')::DOUBLE AS max_min_charge_diff,
      'Discounted Cash Price' AS discounted_price
    FROM 'https://onefact.github.io/datathinking.org-codespace/data/data.payless.health/processed/132655001_Bellevue%20Hospital%20Center_standardcharges.parquet'


,Count


In [14]:
%%sql 

CREATE TABLE IF NOT EXISTS prices AS
    SELECT
      'Discounted Cash Price' AS cash_price
    FROM 'https://onefact.github.io/datathinking.org-codespace/data/data.payless.health/processed/132655001_Bellevue%20Hospital%20Center_standardcharges.parquet'

ConversionException: Conversion Error: Could not convert string 'Discounted Cash Price' to DOUBLE

In [13]:
%%sql 

DESCRIBE bellevue

,column_name,column_type,null,key,default,extra
0,cash_price,VARCHAR,YES,NaN,NaN,NaN


# Mayo Clinic

In [1]:
!head /Users/me/Downloads/hospital_price_transparency_240010_410944601_mayo-clinic-hospital-rochester_standardcharges.csv

"entity_cd","contract_name","product_name","code","code_type","code_desc","area","rate_method","rev_code","rev_desc","hcpcs","implied_quantity","modifier","standard_gross_charge","payer_specific_negotiated_rate","minimum_negotiated","maximum_negotiated","count_of_compared_rates","discounted_cash_price","rate_desc","footnote"
"These are standard charges as of 12-31-2021","","","","","","","","","","","","","","","","","","","",""
"Published date 01-21-2022","","","","","","","","","","","","","","","","","","","",""
"MCH-RST","SCHA-MCHS-Medicaid-20180101","MEDICAID ADV.","'001-1","APR-DRG","LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT","IP","APR-DRG","","","","N/A","","N/A","$54,840.99","$53,303.39","$80,467.62","4","N/A","N/A","11"
"MCH-RST","SCHA-MCR-Medicaid-20180101","MEDICAID ADV.","'001-1","APR-DRG","LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT","IP","APR-DRG","","","","N/A","","N/A","$53,303.39","$53,303.39","$80,467.62","4","N/A","N/A","11"
"MCH-RST","UCare-MCHS-Medicaid-20190101"

In [11]:
%%sql 

SELECT * FROM read_csv('/Users/me/Downloads/hospital_price_transparency_240010_410944601_mayo-clinic-hospital-rochester_standardcharges.csv', 
                       delim=',', 
                       header=True, 
                       quote='"', 
                       nullstr='N/A',
                       -- skip=3,
                       columns={
                           'entity_cd': 'VARCHAR',
                           'contract_name': 'VARCHAR',
                           'product_name': 'VARCHAR',
                           'code': 'VARCHAR',
                           'code_type': 'VARCHAR',
                           'code_desc': 'VARCHAR',
                           'area': 'VARCHAR',
                           'rate_method': 'VARCHAR',
                           'rev_code': 'VARCHAR',
                           'rev_desc': 'VARCHAR',
                           'hcpcs': 'VARCHAR',
                           'implied_quantity': 'VARCHAR',
                           'modifier': 'VARCHAR',
                           'standard_gross_charge': 'VARCHAR',
                           'payer_specific_negotiated_rate': 'VARCHAR',
                           'minimum_negotiated': 'VARCHAR',
                           'maximum_negotiated': 'VARCHAR',
                           'count_of_compared_rates': 'VARCHAR',
                           'discounted_cash_price': 'VARCHAR',
                           'rate_desc': 'VARCHAR',
                           'footnote': 'VARCHAR'
                        });


InvalidInputException: Invalid Input Error: Error in file "/Users/me/Downloads/hospital_price_transparency_240010_410944601_mayo-clinic-hospital-rochester_standardcharges.csv" on line 90196: quote should be followed by end of value, end of row or another quote. (  file=/Users/me/Downloads/hospital_price_transparency_240010_410944601_mayo-clinic-hospital-rochester_standardcharges.csv
  delimiter=','
  quote='"'
  escape='"' (default)
  header=1
  sample_size=20480
  ignore_errors=0
  all_varchar=0). 

# Visualization

In [16]:
import vegafusion as vf
import polars as pl
import altair as alt
import altair as alt
alt.data_transformers.disable_max_rows()
alt.renderers.enable('html')

# Configure DuckDB connection
vf.runtime.set_connection("duckdb")

# Enable Mime Renderer
vf.enable(row_limit=100000000)

vegafusion.enable(mimetype='html', row_limit=100000000, embed_options=None)

In [17]:
# Load the phone calls data into a Polars datafram
bellevue = pl.read_parquet("./data/data.payless.health/processed/132655001_Bellevue Hospital Center_standardcharges.parquet")

In [18]:
bellevue.schema

{'Procedure': Utf8,
 'Code Type': Utf8,
 'Code': Utf8,
 'NDC': Utf8,
 'Rev Code': Utf8,
 'Procedure Description': Utf8,
 'Payer': Utf8,
 'Plan(s)': Utf8,
 'IP Price': Utf8,
 'OP Price': Utf8,
 'Discounted Cash Price': Float64,
 'IP Negotiated Charge': Utf8,
 'OP Negotiated Charge': Utf8,
 'PB Negotiated Charge': Utf8,
 'De-Identified Minimum Negotiated Charge (IP)': Float64,
 'De-Identified Maximum Negotiated Charge (IP)': Float64,
 'De-Identified Minimum Negotiated Charge (OP)': Float64,
 'De-Identified Maximum Negotiated Charge (OP)': Float64}

In [28]:
# Filter out rows that don't have discounted cash price
bellevue = bellevue.filter(
    pl.col("Discounted Cash Price").is_not_null() & pl.col("De-Identified Maximum Negotiated Charge (IP)").is_not_null() & pl.col("De-Identified Minimum Negotiated Charge (IP)").is_not_null()
)

In [29]:
bellevue

Procedure,Code Type,Code,NDC,Rev Code,Procedure Description,Payer,Plan(s),IP Price,OP Price,Discounted Cash Price,IP Negotiated Charge,OP Negotiated Charge,PB Negotiated Charge,De-Identified Minimum Negotiated Charge (IP),De-Identified Maximum Negotiated Charge (IP),De-Identified Minimum Negotiated Charge (OP),De-Identified Maximum Negotiated Charge (OP)
str,str,str,str,str,str,str,str,str,str,f64,str,str,str,f64,f64,f64,f64
"""MS001""","""DRG""","""MS-DRG V39 (FY…","""""","""""","""Heart Transpla…","""EMBLEMHEALTH C…","""EMBLEMHEALTH H…","""1,160,726.62""","""Not Applicable…",464290.65,"""No price histo…","""Not Reimbursed…","""Not Reimbursed…",377996.07,377996.07,null,null
"""MS001""","""DRG""","""MS-DRG V39 (FY…","""""","""""","""Heart Transpla…","""HEALTHFIRST MC…","""HEALTHFIRST PE…","""1,159,323.54""","""Not Applicable…",463729.42,"""No price histo…","""Not Reimbursed…","""Not Reimbursed…",377996.07,377996.07,null,null
"""MS001""","""DRG""","""MS-DRG V39 (FY…","""""","""""","""Heart Transpla…","""HEALTHFIRST MC…","""HEALTHFIRST 65…","""1,159,323.72""","""Not Applicable…",463729.49,"""No price histo…","""Not Reimbursed…","""Not Reimbursed…",377996.07,377996.07,null,null
"""MS001""","""DRG""","""MS-DRG V39 (FY…","""""","""""","""Heart Transpla…","""METROPLUS COMM…","""METROPLUS ESSE…","""1,159,332.68""","""Not Applicable…",463733.07,"""No price histo…","""Not Reimbursed…","""Not Reimbursed…",377996.07,377996.07,null,null
"""MS001""","""DRG""","""MS-DRG V39 (FY…","""""","""""","""Heart Transpla…","""METROPLUS MCD …","""METROPLUS MLTC…","""1,159,332.68""","""Not Applicable…",463733.07,"""No price histo…","""Not Reimbursed…","""Not Reimbursed…",377996.07,377996.07,null,null
"""MS001""","""DRG""","""MS-DRG V39 (FY…","""""","""""","""Heart Transpla…","""UNITEDHEALTHCA…","""UHC COMMERCIAL…","""1,159,332.68""","""Not Applicable…",463733.07,"""No price histo…","""Not Reimbursed…","""Not Reimbursed…",377996.07,377996.07,null,null
"""MS003""","""DRG""","""MS-DRG V39 (FY…","""""","""""","""Ecmo Or Trache…","""AFFINITY HEALT…","""AFFINITY MEDIC…","""628,653.78""","""Not Applicable…",251461.51,"""135,145.45""","""Not Reimbursed…","""Not Reimbursed…",135078.13,406531.89,null,null
"""MS003""","""DRG""","""MS-DRG V39 (FY…","""""","""""","""Ecmo Or Trache…","""EMBLEMHEALTH C…","""EMBLEMHEALTH H…","""607,745.53""","""Not Applicable…",243098.21,"""191,963.26""","""Not Reimbursed…","""Not Reimbursed…",135078.13,406531.89,null,null
"""MS003""","""DRG""","""MS-DRG V39 (FY…","""""","""""","""Ecmo Or Trache…","""EMBLEMHEALTH C…","""EMBLEMHEALTH H…","""607,745.53""","""Not Applicable…",243098.21,"""191,963.26""","""Not Reimbursed…","""Not Reimbursed…",135078.13,406531.89,null,null


In [30]:
# Create a histogram of Discounted Cash Price
alt.Chart(bellevue.to_arrow()).mark_bar().encode(
    alt.X("Discounted Cash Price:Q", bin=True),
    y='count()'
)

alt.Chart(...)

In [31]:
# Create a histogram of Discounted Cash Price
alt.Chart(bellevue.to_arrow()).mark_bar().encode(
    alt.X("De-Identified Minimum Negotiated Charge (OP):Q", bin=True),
    y='count()'
)

thread '<unnamed>' panicked at 'Failed to get node value: InternalError("Cannot convert NULL to f64", ErrorContext { contexts: [] })', /Users/runner/work/vegafusion/vegafusion/vegafusion-runtime/src/task_graph/runtime.rs:775:18


PanicException: Failed to get node value: InternalError("Cannot convert NULL to f64", ErrorContext { contexts: [] })

In [32]:
# Create a histogram of Discounted Cash Price
alt.Chart(bellevue.to_arrow()).mark_bar().encode(
    alt.X("De-Identified Maximum Negotiated Charge (OP):Q", bin=True),
    y='count()'
)

thread '<unnamed>' panicked at 'Failed to get node value: InternalError("Cannot convert NULL to f64", ErrorContext { contexts: [] })', /Users/runner/work/vegafusion/vegafusion/vegafusion-runtime/src/task_graph/runtime.rs:775:18


PanicException: Failed to get node value: InternalError("Cannot convert NULL to f64", ErrorContext { contexts: [] })

In [33]:
# Create a histogram of Discounted Cash Price
alt.Chart(bellevue.to_arrow()).mark_bar().encode(
    alt.X("De-Identified Minimum Negotiated Charge (IP):Q", bin=True),
    y='count()'
)

alt.Chart(...)

In [34]:
# Create a histogram of Discounted Cash Price
alt.Chart(bellevue.to_arrow()).mark_bar().encode(
    alt.X("De-Identified Maximum Negotiated Charge (IP):Q", bin=True),
    y='count()'
)

alt.Chart(...)

In [35]:
bellevue.write_parquet("./data/data.payless.health/processed/132655001_Bellevue Hospital Center_standardcharges_filtered.parquet")